# STAT3009 Project 1
- LAW Yiu Leung Eric       1155149315
- LAM Wai Chiu          1155152095

## Import packages and load the developed methods

In [1]:
import numpy as np
import pandas as pd

from developed_methods import *

## Pre-process the raw data
- check the `user_id` and `item_id`: mapping `item_id` to a continuous sequence based on `sklean.preprocessing`
- use `sklearn.model_selection.train_test_split` to generate `train` and `test` dataset
- create `train_pair`,`train_rating`, `test_pair`

In [2]:
dtrain = pd.read_csv("data/train.csv")
dtest = pd.read_csv("data/test.csv")

## mapping 
from sklearn import preprocessing
le_user = preprocessing.LabelEncoder()
le_user.fit(np.append(dtrain['user_id'], dtest["user_id"]))
dtrain['user_id'] = le_user.transform(dtrain["user_id"])
dtest["user_id"] = le_user.transform(dtest["user_id"])

le_item = preprocessing.LabelEncoder()
le_item.fit(np.append(dtrain['item_id'], dtest["item_id"]))
dtrain["item_id"] = le_item.transform(dtrain["item_id"])
dtest["item_id"] = le_item.transform(dtest["item_id"])

Create train_pair, train_rating and test_pair

In [3]:
# train_pair, train_rating
train_pair = dtrain[['user_id', 'item_id']].values
train_rating = dtrain['rating'].values

# test_pair
test_pair = dtest[['user_id', 'item_id']].values
n_user, n_item = max(train_pair[:,0].max(), test_pair[:,0].max())+1, max(train_pair[:,1].max(), test_pair[:,1].max())+1

# Impelement deep learning with Keras

In [4]:
import pandas as pd
import numpy as np
from zipfile import ZipFile
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from pathlib import Path
import matplotlib.pyplot as plt

## Create the model
We embed both users and movies in to 50-dimensional vectors.

The model computes a match score between user and movie embeddings via a dot product, and adds a per-movie and per-user bias.

In [5]:
class LFactorNet(keras.Model):
    def __init__(self, num_users, num_items, embedding_size, **kwargs):
        super(LFactorNet, self).__init__(**kwargs)
        self.num_users = num_users
        self.num_items = num_items
        self.embedding_size = embedding_size
        self.user_embedding = layers.Embedding(
            num_users,
            embedding_size,
            embeddings_initializer="he_normal",
            embeddings_regularizer=keras.regularizers.l2(1e-2),
        )
        self.user_bias = layers.Embedding(num_users, 1)
        self.glb_bias = tf.Variable(0., trainable=True) 
        self.movie_embedding = layers.Embedding(
            num_items,
            embedding_size,
            embeddings_initializer="he_normal",
            embeddings_regularizer=keras.regularizers.l2(1e-2),
        )
        self.movie_bias = layers.Embedding(num_items, 1)

    def call(self, inputs):
        user_vector = self.user_embedding(inputs[:, 0])
        user_bias = self.user_bias(inputs[:, 0])
        movie_vector = self.movie_embedding(inputs[:, 1])
        movie_bias = self.movie_bias(inputs[:, 1])
        dot_user_movie = tf.tensordot(user_vector, movie_vector, 2)
        # Add all the components (including bias)
        x = dot_user_movie + user_bias + movie_bias + self.glb_bias
        return x

In [6]:
## Baseline + NCF
# glb mean
glb_ave = glb_mean()
glb_ave.fit(train_rating)
pred = glb_ave.predict(test_pair)

# user_mean
train_rating_res = train_rating - glb_ave.predict(train_pair)
user_ave = user_mean(n_user=n_user)
user_ave.fit(train_pair=train_pair, train_ratings=train_rating_res)
pred = pred + user_ave.predict(test_pair)

# item_mean
train_rating_res -=  user_ave.predict(train_pair)
item_ave = item_mean(n_item=n_item)
item_ave.fit(train_pair=train_pair, train_ratings=train_rating_res)
pred = pred + item_ave.predict(test_pair)

train_rating_res -= item_ave.predict(train_pair)

In [7]:
model = LFactorNet(num_users=n_user, num_items=n_item, embedding_size=50)


metrics = [
    keras.metrics.MeanAbsoluteError(name='mae'),
    keras.metrics.RootMeanSquaredError(name='rmse')
]

model.compile(
    optimizer=keras.optimizers.Adam(1e-3), 
    loss=tf.keras.losses.MeanSquaredError(), 
    metrics=metrics
)

callbacks = [keras.callbacks.EarlyStopping( 
    monitor='val_rmse', min_delta=0, patience=5, verbose=1, 
    mode='auto', baseline=None, restore_best_weights=True)]

history = model.fit(
    x=train_pair,
    y=train_rating_res,
    batch_size=64,
    epochs=100,
    verbose=1,
    validation_split=.2,
)

Epoch 1/100
192/192 [==============================] - 2s 4ms/step - loss: 2.0427 - mae: 0.9057 - rmse: 1.1670 - val_loss: 1.5977 - val_mae: 0.9038 - val_rmse: 1.1702
Epoch 2/100
192/192 [==============================] - 0s 2ms/step - loss: 1.4557 - mae: 0.8999 - rmse: 1.1611 - val_loss: 1.4107 - val_mae: 0.9050 - val_rmse: 1.1724
Epoch 3/100
192/192 [==============================] - 0s 2ms/step - loss: 1.3569 - mae: 0.8971 - rmse: 1.1581 - val_loss: 1.3859 - val_mae: 0.9075 - val_rmse: 1.1752
Epoch 4/100
192/192 [==============================] - 0s 2ms/step - loss: 1.3368 - mae: 0.8949 - rmse: 1.1554 - val_loss: 1.3876 - val_mae: 0.9093 - val_rmse: 1.1778
Epoch 5/100
192/192 [==============================] - 0s 2ms/step - loss: 1.3292 - mae: 0.8926 - rmse: 1.1528 - val_loss: 1.3939 - val_mae: 0.9117 - val_rmse: 1.1806
Epoch 6/100
192/192 [==============================] - 0s 2ms/step - loss: 1.3239 - mae: 0.8907 - rmse: 1.1506 - val_loss: 1.4001 - val_mae: 0.9139 - val_rmse: 1.183

## Make prediction

In [19]:
pred_rating = pred + model.predict(test_pair).flatten()
pred_rating = np.array(pred_rating)
print(pred_rating)

[2.82646314 4.63462502 5.00327688 ... 3.4511113  9.73770997 4.15949607]


## Adjust the prediction within [0, 15]

In [23]:
def adjust(pred_rating, score_min, score_max):
    pred_rating_adjusted = pred_rating.copy()
    pred_rating_adjusted[pred_rating > score_max] = score_max
    pred_rating_adjusted[pred_rating < score_min] = score_min
    return pred_rating_adjusted

In [24]:
pred_rating = adjust(pred_rating, 0, 15)

submit = pd.DataFrame({"id": range(len(dtest)), "rating": pred_rating})
submit.to_csv("predict/NCF.csv", index = False)